In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
np.random.seed(123)

import utils

In [ ]:
train_file = '../data/input/train.csv'
df_data = pd.read_csv(train_file)
df_data.drop(['ID_code'],axis=1, inplace=True)

target = df_data.pop('target')

In [ ]:
def add_features_save_para(df, data='train', feature_dict = None):
    if data=='train':
        feature_dict = {}
    # Add more features
    for col in df.columns:
        # Normalize the data, so that it can be used in norm.cdf(), as though it is a standard normal 
        # variable
        if data=='train':
            mean = df[col].mean()
            std = df[col].std()
            df[col] = ((df[col] - mean)/std).astype('float32')
            feature_dict[col] = {'mean': mean, 'std': std} 
        else:
            df[col] = ((df[col] - feature_dict[col]['mean'])/feature_dict[col]['std']).astype('float32')
            
        # Square
        df[col+'_s'] = df[col] * df[col]

        # Cube
        df[col+'_c'] = df[col] * df[col] * df[col]

        # 4th power
        df[col+'_q'] = df[col] * df[col] * df[col] * df[col]

        # Cumulative percentile (not normalized)
        df[col+'_r'] = rankdata(df[col]).astype('float32')

        # Cumulative normal percentile
        df[col+'_n'] = norm.cdf(df[col]).astype('float32')



    # Normalize the data. Again.
    for col in df.columns:
        df[col] = ((df[col] - df[col].mean())/df[col].std()).astype('float32')
        
    return df, feature_dict

In [ ]:
df_train, features_dict = add_features_save_para(df_data)

In [ ]:
test_file = '../data/input/test.csv'
df_test = pd.read_csv(test_file)
ID = df_test.pop('ID_code')

In [ ]:
df_test, features_dict = add_features_save_para(df_test, data='test', feature_dict=features_dict)

In [ ]:
df_train['target'] = target

In [ ]:
df_data = pd.concat([df_train, df_test])

In [ ]:
utils.to_pickles(df_data, '../data/input/train_test_normed_featrs_mod_one')